In [103]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_C", "B_P", "I_C", "I_P"]
ner_args = NERArgs()
ner_args.max_seq_length = 512
test_ner = NERModel("bert", 'model_seq_512/save_model', args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS)

In [104]:
def tag_html_format(pred_list):
    LIST_TAGS = ['claim', 'premise', 'o']
    REPRESEN_TAGS = ['c', 'p', 'O']
    text_convert = ''
    prev_tag = ''
    trigger_tag = False 


    for word, label in pred_list:
        tags = label.split('-')
        next_tag = tags[0] if len(tags) == 1 else tags[1]
        
        if prev_tag != next_tag:
            if prev_tag:
                text_convert += '</' + html_tag + '>'

            html_tag = LIST_TAGS[REPRESEN_TAGS.index(next_tag)]
            prev_tag = next_tag
            trigger_tag = not(trigger_tag)

            if trigger_tag:
                text_convert += '<' + html_tag + '>'
            else:
                text_convert += '<' + html_tag + '>'

        text_convert += word
    text_convert += '</'+ LIST_TAGS[REPRESEN_TAGS.index(prev_tag)]+'>'
    text_convert = text_convert.replace('<o>', '').replace('</o>', '')
            
    return text_convert

In [105]:
def predict_text(text):
    predict_text = []
    text_token = word_tokenize(text)
    predictions, raw_outputs = test_ner.predict([text_token], split_on_space=False)
    # convert [{}] to [()]
    # convert B_C to B-c
    for pred_dict in predictions[0]:
        for word, label in pred_dict.items():
             label = label.capitalize().replace('_', '-')
             predict_text.append((word, label))
    return predict_text, tag_html_format(predict_text)

In [106]:
from pythainlp.tokenize import word_tokenize

text = "การใช้ถุงพลาสติดแทนถุงผ้าผมคิดว่ามันจะทำให้ช่วยลดโลกร้อนได้มากขึ้น"

pred_text, pred_tag = predict_text(text)
print(pred_text)
print("----")
print(pred_tag)

Running Prediction: 100%|██████████| 1/1 [00:02<00:00,  2.25s/it]

[('การ', 'B-c'), ('ใช้', 'I-c'), ('ถุง', 'I-c'), ('พ', 'I-c'), ('ลา', 'I-c'), ('สติ', 'I-c'), ('ด', 'I-c'), ('แทน', 'I-c'), ('ถุง', 'I-c'), ('ผ้า', 'I-c'), ('ผม', 'I-c'), ('คิด', 'I-c'), ('ว่า', 'I-p'), ('มัน', 'I-p'), ('จะ', 'I-p'), ('ทำให้', 'I-p'), ('ช่วย', 'I-p'), ('ลด', 'I-p'), ('โลก', 'I-p'), ('ร้อน', 'I-p'), ('ได้', 'I-p'), ('มากขึ้น', 'I-p')]
----
<claim>การใช้ถุงพลาสติดแทนถุงผ้าผมคิด</claim><premise>ว่ามันจะทำให้ช่วยลดโลกร้อนได้มากขึ้น</premise>
